In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
file_path = "/Users/sandychen/Downloads/Combined_Final.csv"
human_Fabre_labels = pd.read_csv(file_path)

In [125]:
human_Fabre_labels["Final_Label"] = human_Fabre_labels["Risk"].apply(lambda x: 1 if x in ("High Risk", "Intermediate Risk") else 0)
human_Fabre_labels["Final_Result"] = human_Fabre_labels["Result"].apply(lambda x: 1 if x in ("Positive") else 0)

In [3]:
encounter_level = human_Fabre_labels.drop_duplicates(subset=["MRN", "NoteDateTime"])
print(f"Total number of encounters: {len(encounter_level)}")

Total number of encounters: 114


In [126]:
human_Fabre_labels["Result"].value_counts()

Result
Positive    170
Negative     52
Name: count, dtype: int64

In [5]:
encounter_level["Result"].value_counts(normalize=True)

Result
Positive    0.798246
Negative    0.201754
Name: proportion, dtype: float64

## above code checks the result distribution at the encounter level
we have 80% positive, 20% negative from 114 encounter

In [6]:
import pandas as pd
import glob
import os

# Path to your shared drive folder
folder = "/Users/sandychen/Library/CloudStorage/GoogleDrive-wenyuanc@stanford.edu/Shared drives/INFD Projects/Blood Cultures/Adult/Labeled Cases/Phase 1"
# Columns you care about
import pandas as pd
import glob
import os

# Path to your shared drive folder
folder = "/Users/sandychen/Library/CloudStorage/GoogleDrive-wenyuanc@stanford.edu/Shared drives/INFD Projects/Blood Cultures/Adult"

# Map messy/variant names → canonical
rename_map = {
    "blood_culture_taken_time_exact": "BloodCultureOrderDateTime",
    "BloodCultureOrderDateTime": "BloodCultureOrderDateTime",
    "notedatetime_exact": "NoteDateTime",
    "NoteDateTime": "NoteDateTime",
    "Reviewer ": "Reviewer",
    "Tie Breaker Reviewer": "Reviewer",  # optional consolidation
}

# Final columns you want
keep_cols = ["MRN", "BloodCultureOrderDateTime", "NoteDateTime", "NoteType", "Reviewer", "Risk of Bactermia", "source_file"]

all_data = []
xlsx_files = glob.glob(os.path.join(folder, "**", "*.xlsx"), recursive=True)

for file in xlsx_files:
    if "BloodCultureAnalysis_Phase1_Tiebreaker.xlsx" in file:
        continue
    try:
        df = pd.read_excel(file, engine="openpyxl")
        df["source_file"] = os.path.basename(file)

        # Rename columns (normalize known variants)
        df = df.rename(columns={c: rename_map.get(c, c) for c in df.columns})

        # Keep only desired columns (if missing, fill with NA)
        slim = pd.DataFrame()
        for col in keep_cols:
            if col in df.columns:
                slim[col] = df[col]
            else:
                slim[col] = pd.NA

        all_data.append(slim)
        print(f"✅ Processed {file} with {slim.shape[0]} rows")
    except Exception as e:
        print(f"⚠️ Could not read {file}: {e}")

# Combine
if all_data:
    combined = pd.concat(all_data, ignore_index=True)
    print("\nFinal shape:", combined.shape)
    print(combined.head())
else:
    print("No data loaded.")




✅ Processed /Users/sandychen/Library/CloudStorage/GoogleDrive-wenyuanc@stanford.edu/Shared drives/INFD Projects/Blood Cultures/Adult/Labeled Cases/Phase 1/BloodCultureAnalysis_Phase1_Kameron.xlsx with 121 rows
✅ Processed /Users/sandychen/Library/CloudStorage/GoogleDrive-wenyuanc@stanford.edu/Shared drives/INFD Projects/Blood Cultures/Adult/Labeled Cases/Phase 1/BloodCultureAnalysis_Phase1_Unknown.xlsx with 40 rows
✅ Processed /Users/sandychen/Library/CloudStorage/GoogleDrive-wenyuanc@stanford.edu/Shared drives/INFD Projects/Blood Cultures/Adult/Labeled Cases/Phase 1/BloodCultureAnalysis_Phase1_Aydin.xlsx with 120 rows
✅ Processed /Users/sandychen/Library/CloudStorage/GoogleDrive-wenyuanc@stanford.edu/Shared drives/INFD Projects/Blood Cultures/Adult/Labeled Cases/Phase 1/BloodCultureAnalysis_Phase2_NickManoj.xlsx with 401 rows

Final shape: (682, 7)
          MRN BloodCultureOrderDateTime        NoteDateTime     NoteType  \
0  10711356.0       2022-12-07 15:17:00 2022-12-07 15:04:00  E

In [7]:
encounter_level["NoteDateTime"] = pd.to_datetime(encounter_level["NoteDateTime"])
combined["NoteDateTime"] = pd.to_datetime(combined["NoteDateTime"])


/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_96397/1618871322.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  encounter_level["NoteDateTime"] = pd.to_datetime(encounter_level["NoteDateTime"])
/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_96397/1618871322.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  encounter_level["NoteDateTime"] = pd.to_datetime(encounter_level["NoteDateTime"])


In [8]:
combined_encounter = combined[combined["MRN"].isin(encounter_level["MRN"]) & combined["NoteDateTime"].isin(encounter_level["NoteDateTime"])]


In [9]:
combined_encounter["Risk of Bactermia"].value_counts()

Risk of Bactermia
High Risk            71
HighRisk             66
Low Risk             46
Intermediate Risk    44
IntermediateRisk     44
Unclassified         34
LowRisk              21
Unclassified          4
Name: count, dtype: int64

In [10]:
combined_encounter["Final"] = combined_encounter["Risk of Bactermia"].apply(
    lambda x: -1 if pd.isna(x) 
    else 1 if x in ("High Risk", "HighRisk") 
    else 0)

/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_96397/1911870299.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_encounter["Final"] = combined_encounter["Risk of Bactermia"].apply(


In [11]:
combined_encounter_final_label = combined_encounter.groupby(["MRN","NoteDateTime", "Reviewer"])["Final"].max().reset_index(name="final_label")

In [13]:
def fix_missing_reviewer(group):
    labels = group["final_label"].values
    
    # Valid labels = anything not -1
    valid = labels[labels != -1]

    if len(valid) == 0:
        # Drop this group entirely
        return None
    
    if len(valid) == 1:
        # Only one reviewer rated → assign their label to both rows
        group["final_label"] = valid[0]
    # else: if 2 reviewers rated, keep as-is

    return group

fixed = (
    combined_encounter_final_label
    .groupby(["MRN", "NoteDateTime"], group_keys=False)
    .apply(fix_missing_reviewer)
)

# Drop cases where both reviewers failed (-1)
fixed = fixed.dropna(subset=["final_label"])


/var/folders/d1/3gdyy98d6h1d9pxx47s40vv40000gp/T/ipykernel_96397/1299294224.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  combined_encounter_final_label


In [15]:
combined_encounter_final_label_pivot = fixed.pivot_table(
    index=["MRN", "NoteDateTime"],
    columns="Reviewer",
    values="final_label"
).reset_index()


In [16]:
# Count how many reviewers labeled each case
combined_encounter_final_label_pivot["n_raters"] = combined_encounter_final_label_pivot.iloc[:, 2:].notna().sum(axis=1)

# Keep only cases with exactly 2 reviewers
combined_encounter_final_label_pairs = combined_encounter_final_label_pivot[combined_encounter_final_label_pivot["n_raters"] == 2]


In [20]:
label_cols = ["Aydin", "Kameron", "Manoj", "Marshall"]
phys1 = []
phys2 = []

for _, row in combined_encounter_final_label_pairs.iterrows():
    vals = row[label_cols].dropna().tolist()   # the two labels
    phys1.append(vals[0])
    phys2.append(vals[1])



phys1 = np.array(phys1)
phys2 = np.array(phys2)
from sklearn.metrics import cohen_kappa_score
import numpy as np
import warnings

# Compute point estimate
kappa = cohen_kappa_score(phys1, phys2)
print("Kappa:", kappa)

# Bootstrap CI
n = len(phys1)
rng = np.random.default_rng(0)
boot = []

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for _ in range(5000):
        idx = rng.integers(0, n, n)
        boot.append(cohen_kappa_score(phys1[idx], phys2[idx]))

boot = np.array(boot)
CI_low, CI_high = np.percentile(boot, [2.5, 97.5])

print("95% CI:", CI_low, CI_high)



Kappa: 0.7327289778055642
95% CI: 0.5977293648358393 0.8531873792659369


# human confusion metric analysis

In [90]:
human_Fabre_labels["Risk"].value_counts()

Risk
High Risk            101
Intermediate Risk     82
Low Risk              35
Unclassified           4
Name: count, dtype: int64

In [91]:
human_Fabre_labels["Final_Label"] = human_Fabre_labels["Risk"].apply(lambda x: 1 if x in ("High Risk", "Intermediate Risk") else 0)
human_Fabre_labels["Final_Result"] = human_Fabre_labels["Result"].apply(lambda x: 1 if x in ("Positive") else 0)

In [92]:
human_Fabre_labels

,MRN,anon_ID,jitter,BloodCultureOrderDateTime,NoteDateTime,NoteType,Risk,Result,Result - Full,Final_Label,Final_Result
0,10711356,NaN,NaN,12/7/22 15:17,12/7/22 15:04,ED Provider,Low Risk,Negative,No growth,0,0
1,10711356,NaN,NaN,12/7/22 15:55,12/7/22 15:04,ED Provider,Low Risk,Negative,No growth,0,0
2,84625987,NaN,NaN,1/11/23 14:14,1/11/23 13:54,ED Provider,High Risk,Negative,No growth,1,0
3,84625987,NaN,NaN,1/11/23 14:47,1/11/23 13:54,ED Provider,High Risk,Negative,No growth,1,0
4,17081944,NaN,NaN,3/29/23 8:12,3/29/23 8:13,ED Provider,Low Risk,Negative,No growth,0,0
...,...,...,...,...,...,...,...,...,...,...,...
217,7171226,NaN,NaN,1/20/23 2:08,1/20/23 1:37,ED Provider,Low Risk,Positive,Lactose fermenting GNR (E coli),0,1
218,23828635,NaN,NaN,1/31/23 15:25,1/31/23 16:32,ED Provider,High Risk,Positive,S. aureus,1,1
219,23828635,NaN,NaN,1/31/23 15:57,1/31/23 16:32,ED Provider,High Risk,Positive,S. aureus,1,1
220,82062696,NaN,NaN,2/26/23 20:58,2/26/23 20:01,ED Provider,Low Risk,Negative,No growth,0,0


In [93]:
human_Fabre_labels_encounter_level = (
    human_Fabre_labels
    .groupby(["MRN", "NoteDateTime"])[["Final_Label", "Final_Result", "BloodCultureOrderDateTime"]]
    .max()
    .reset_index()
)


In [116]:
human_Fabre_labels_encounter_level["Final_Result"].value_counts()

Final_Result
1    103
0     11
Name: count, dtype: int64

In [94]:
len(human_Fabre_labels_encounter_level["MRN"].tolist())

114

In [95]:
from sklearn.metrics import confusion_matrix

y_true = human_Fabre_labels_encounter_level["Final_Label"].astype(int)
y_pred = human_Fabre_labels_encounter_level["Final_Result"].astype(int)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
TN, FP, FN, TP = cm.ravel()

# Metrics
sensitivity = TP / (TP + FN) if (TP + FN) > 0 else float('nan')
specificity = TN / (TN + FP) if (TN + FP) > 0 else float('nan')
precision = TP / (TP + FP) if (TP + FP) > 0 else float('nan')
npv = TN / (TN + FN) if (TN + FN) > 0 else float('nan')
f1 = 2*TP / (2*TP + FP + FN) if (2*TP + FP + FN) > 0 else float('nan')
accuracy = (TP + TN) / (TP + TN + FP + FN)
balanced_accuracy = (sensitivity + specificity) / 2

# Print results
print("Confusion Matrix:")
print(cm)
print(f"TN={TN}, FP={FP}, FN={FN}, TP={TP}\n")

print(f"Sensitivity (Recall):         {sensitivity:.3f}")
print(f"Specificity:                  {specificity:.3f}")
print(f"Precision (PPV):             {precision:.3f}")
print(f"Negative Predictive Value:   {npv:.3f}")
print(f"F1 Score:                     {f1:.3f}")
print(f"Accuracy:                     {accuracy:.3f}")
print(f"Balanced Accuracy:           {balanced_accuracy:.3f}")


Confusion Matrix:
[[ 7 13]
 [ 4 90]]
TN=7, FP=13, FN=4, TP=90

Sensitivity (Recall):         0.957
Specificity:                  0.350
Precision (PPV):             0.874
Negative Predictive Value:   0.636
F1 Score:                     0.914
Accuracy:                     0.851
Balanced Accuracy:           0.654


In [127]:
human_Fabre_labels_encounter_level

,MRN,NoteDateTime,Final_Label,Final_Result,BloodCultureOrderDateTime
0,1039585,12/21/22 23:17,1,1,12/21/22 21:07
1,1071141,1/14/23 19:25,1,1,1/14/23 20:55
2,3187432,12/17/22 15:08,1,1,12/17/22 16:37
3,4922779,2/10/23 12:43,0,1,2/10/23 14:41
4,5569918,12/13/22 0:57,1,1,12/13/22 4:32
...,...,...,...,...,...
109,84970912,1/24/23 22:31,1,1,1/24/23 19:16
110,85080398,2/8/23 18:15,0,1,2/8/23 19:56
111,85258846,2/17/23 21:59,1,1,2/18/23 0:30
112,85386969,3/18/23 14:24,1,1,3/18/23 14:51


In [96]:
human_Fabre_labels_encounter_level.sort_values(by="MRN", ascending=True)

,MRN,NoteDateTime,Final_Label,Final_Result,BloodCultureOrderDateTime
0,1039585,12/21/22 23:17,1,1,12/21/22 21:07
1,1071141,1/14/23 19:25,1,1,1/14/23 20:55
2,3187432,12/17/22 15:08,1,1,12/17/22 16:37
3,4922779,2/10/23 12:43,0,1,2/10/23 14:41
4,5569918,12/13/22 0:57,1,1,12/13/22 4:32
...,...,...,...,...,...
109,84970912,1/24/23 22:31,1,1,1/24/23 19:16
110,85080398,2/8/23 18:15,0,1,2/8/23 19:56
111,85258846,2/17/23 21:59,1,1,2/18/23 0:30
112,85386969,3/18/23 14:24,1,1,3/18/23 14:51


In [128]:
human_Fabre_labels_encounter_level["MRN"].nunique()

112

In [35]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
%load_ext google.cloud.bigquery
from google.cloud import bigquery

client=bigquery.Client()
project_id = "som-nero-phi-jonc101"

/Users/sandychen/Desktop/Healthrex_workspace/scripts/Blood_Culture_Stewardship/blood_culture_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/sandychen/Desktop/Healthrex_workspace/scripts/Blood_Culture_Stewardship/blood_culture_env/lib/python3.9/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


In [44]:
mrn_list = set(human_Fabre_labels_encounter_level["MRN"].tolist())
mrn_list_str = ",".join(str(x) for x in mrn_list)


In [145]:
query = f"""
WITH  demo AS (
select * from `som-nero-phi-jonc101-secure.starr_map.shc_map_2024-11-17` as c
where  CAST(c.mrn AS INT64) IN ({mrn_list_str})),

note AS (
      SELECT note.*,
      demo.mrn,
      demo.jitter
      FROM `som-nero-phi-jonc101.Deid_Notes_JChen.Deid_Notes_SHC_JChen` AS note
      JOIN `som-nero-phi-jonc101.shc_core_2024.prov_map` AS m
        ON m.shc_prov_id = CAST(SUBSTR(note.author_prov_map_id, 2) AS STRING)
        JOIN demo
        on demo.anon_id = note.anon_id
      WHERE note.note_type_desc = 'ED Provider Notes'
        AND m.prov_type = 'PHYSICIAN'


  ),
  merged AS(SELECT
      c.*,  
      # c.anon_id,
      # c.pat_enc_csn_id_coded,
      # c.order_proc_id_coded,
      # c.blood_culture_order_datetime_utc,
      n.jittered_note_date AS notedatetime_not_utc,
      n.jittered_note_date_utc AS notedatetime,
      n.mrn,
      n.jitter,
      n.deid_note_text
  FROM som-nero-phi-jonc101.blood_culture_stewardship_sandy_2024.enriched_label_filtered_adult_only_analysis_cohort_all_features c
  JOIN note AS n
    ON c.anon_id = n.anon_id
  WHERE TIMESTAMP_DIFF(n.jittered_note_date_utc,
                        TIMESTAMP(c.blood_culture_order_datetime_utc),
                        HOUR)
        BETWEEN -24 AND 24)
        
  select m.*, 
  TIMESTAMP_SUB(TIMESTAMP_SUB(m.blood_culture_order_datetime_utc, INTERVAL m.jitter DAY), INTERVAL 8 hour) as unjittered_blood_culture_order_datetime,
  TIMESTAMP_SUB(m.notedatetime_not_utc, INTERVAL m.jitter DAY) as unjittered_notedatetime
  from merged m
        """
  


valid_notes = client.query(query).to_dataframe()

In [146]:
valid_notes["mrn"] = valid_notes["mrn"].astype(int)
# Make sure NoteDateTime is actually datetime
human_Fabre_labels_encounter_level["NoteDateTime"] = pd.to_datetime(
    human_Fabre_labels_encounter_level["NoteDateTime"]
)

# (Optional) also ensure unjittered_notedatetime is datetime, if not already
valid_notes["unjittered_notedatetime"] = pd.to_datetime(
    valid_notes["unjittered_notedatetime"]
)

# Now merge
merged = valid_notes.merge(
    human_Fabre_labels_encounter_level,
    left_on=["mrn", "unjittered_notedatetime"],
    right_on=["MRN", "NoteDateTime"],
    how="inner",
)


In [147]:
human_Fabre_labels_encounter_level[~human_Fabre_labels_encounter_level["NoteDateTime"].isin(merged["NoteDateTime"])]

,MRN,NoteDateTime,Final_Label,Final_Result,BloodCultureOrderDateTime
34,18405456,2022-12-19 17:18:00,0,1,12/19/22 18:34


# LLM evaluation on the same test set (114)

In [159]:
path = "../data/EHR_IncludeInprompt_True_fabre_minimal/blood_culture_llm_predictions_fabre_minimal_20251205_132638.csv"
llm_predictions = pd.read_csv(path)

In [185]:
path = "../data/EHR_IncludeInprompt_True_fabre_minimal_v2/blood_culture_llm_predictions_fabre_minimal_v2_20251205_152747.csv"
llm_predictions = pd.read_csv(path)

In [186]:
llm_predictions["final_label"].value_counts()

final_label
positive    102
negative     10
Name: count, dtype: int64

In [188]:
llm_predictions

,anon_id,pat_enc_csn_id_coded,order_proc_id_coded,blood_culture_order_datetime_utc,final_label,classification,rationale
0,JC3218426,131347572748,848944865,2023-01-06 03:52:00+00:00,positive,INTERMEDIATE,"This is an adult, non-neutropenic patient with..."
1,JC1538188,131347081585,846583067,2023-02-05 21:44:00+00:00,positive,HIGH,"This adult, non-neutropenic patient has explic..."
2,JC6414893,131349733752,858544909,2023-03-23 00:38:00+00:00,positive,LOW,"Adult, non-neutropenic patient with CT evidenc..."
3,JC2957662,131347203814,847097733,2023-01-12 08:46:00+00:00,positive,HIGH,"Adult, non-neutropenic patient with explicit s..."
4,JC557579,131348880755,855224383,2023-01-21 20:55:00+00:00,positive,LOW,"Adult, non‑neutropenic patient without explici..."
...,...,...,...,...,...,...,...
107,JC1944721,131344800694,835780828,2023-01-07 20:19:00+00:00,positive,HIGH,"Per Fabre et al., severe sepsis or septic shoc..."
108,JC1657342,131351669060,865783407,2023-03-08 05:09:00+00:00,positive,HIGH,Adult non-neutropenic patient (ANC 6.64) with ...
109,JC655780,131345315635,838185421,2023-01-08 03:32:00+00:00,positive,INTERMEDIATE,"This adult, non-neutropenic patient has system..."
110,JC728137,131346649372,843930919,2023-02-04 22:34:00+00:00,positive,LOW,"Adult, non-neutropenic patient without explici..."


In [190]:
llm_predictions["classification"].value_counts()

classification
INTERMEDIATE    46
HIGH            33
LOW             32
UNDETERMINED     1
Name: count, dtype: int64

In [192]:
llm_predictions["final_prediction"] = llm_predictions["classification"].apply(lambda x: 1 if x in ("HIGH", "INTERMEDIATE") else 0)

In [193]:
llm_predictions["final_label_result_bin"] = llm_predictions["final_label"].apply(lambda x: 1 if x in ("positive")else 0)

In [195]:
from sklearn.metrics import confusion_matrix

y_true = llm_predictions["final_label_result_bin"].astype(int)
y_pred = llm_predictions["final_prediction"].astype(int)

# Confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
TN, FP, FN, TP = cm.ravel()

# Metrics
sensitivity = TP / (TP + FN) if (TP + FN) > 0 else float('nan')
specificity = TN / (TN + FP) if (TN + FP) > 0 else float('nan')
precision = TP / (TP + FP) if (TP + FP) > 0 else float('nan')
npv = TN / (TN + FN) if (TN + FN) > 0 else float('nan')
f1 = 2*TP / (2*TP + FP + FN) if (2*TP + FP + FN) > 0 else float('nan')
accuracy = (TP + TN) / (TP + TN + FP + FN)
balanced_accuracy = (sensitivity + specificity) / 2

# Print results
print("Confusion Matrix:")
print(cm)
print(f"TN={TN}, FP={FP}, FN={FN}, TP={TP}\n")

print(f"Sensitivity (Recall):         {sensitivity:.3f}")
print(f"Specificity:                  {specificity:.3f}")
print(f"Precision (PPV):             {precision:.3f}")
print(f"Negative Predictive Value:   {npv:.3f}")
print(f"F1 Score:                     {f1:.3f}")
print(f"Accuracy:                     {accuracy:.3f}")
print(f"Balanced Accuracy:           {balanced_accuracy:.3f}")


Confusion Matrix:
[[ 4  6]
 [29 73]]
TN=4, FP=6, FN=29, TP=73

Sensitivity (Recall):         0.716
Specificity:                  0.400
Precision (PPV):             0.924
Negative Predictive Value:   0.121
F1 Score:                     0.807
Accuracy:                     0.688
Balanced Accuracy:           0.558


# Kappa between LLM and physician labels (not actual results)

In [196]:
valid_notes_sub = valid_notes[["anon_id","pat_enc_csn_id_coded","order_proc_id_coded","mrn","unjittered_notedatetime"]]

In [197]:
llm_prediction_physician_labels = valid_notes_sub.merge(human_Fabre_labels_encounter_level, left_on=["mrn","unjittered_notedatetime"], right_on=["MRN","NoteDateTime"], how="inner")

In [198]:
llm_prediction_with_physician_label = llm_prediction_physician_labels.merge(llm_predictions, on=["anon_id","pat_enc_csn_id_coded","order_proc_id_coded"], how="right")

In [200]:
phys1 = llm_prediction_with_physician_label["final_prediction"].tolist()
phys2 = llm_prediction_with_physician_label["Final_Label"].tolist()




phys1 = np.array(phys1)
phys2 = np.array(phys2)
from sklearn.metrics import cohen_kappa_score
import numpy as np
import warnings

# Compute point estimate
kappa = cohen_kappa_score(phys1, phys2)
print("Kappa:", kappa)

# Bootstrap CI
n = len(phys1)
rng = np.random.default_rng(0)
boot = []

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    for _ in range(5000):
        idx = rng.integers(0, n, n)
        boot.append(cohen_kappa_score(phys1[idx], phys2[idx]))

boot = np.array(boot)
CI_low, CI_high = np.percentile(boot, [2.5, 97.5])

print("95% CI:", CI_low, CI_high)



Kappa: 0.4118161925601751
95% CI: 0.2175128085700978 0.5955962845565647
